In [ ]:
pip install keras-layer-normalization

  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.14.0-cp36-none-any.whl size=5268 sha256=42dcae3a5ca3ca260ff27433612efad3ba12319cc99d8e444303a97d11b5df23
  Stored in directory: /root/.cache/pip/wheels/54/80/22/a638a7d406fd155e507aa33d703e3fa2612b9eb7bb4f4fe667
Successfully built keras-layer-normalization


In [ ]:
import os, sys
import pickle
import numpy as np
import matplotlib       
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.models import Model, load_model, Sequential
from keras.layers import Input, LSTM, Bidirectional, Dropout, Dense, Activation,TimeDistributed
from keras import regularizers
from keras_layer_normalization import LayerNormalization
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [ ]:
#Mounting drive: 
from google.colab import drive
drive.mount('/content/drive1')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive1


In [ ]:
#Training Dataset
pickle_labels = open('/content/drive1/My Drive/Training_Set_labels1.pickle',"rb")
labels_train = pickle.load(pickle_labels)
print(np.array(labels_train).shape)
print(np.array(labels_train))

pickle_values = open('/content/drive1/My Drive/Training_Set_values1.pickle',"rb")
values_train = pickle.load(pickle_values)
print(np.array(values_train).shape)

(9508, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
(9508, 8, 2048)


In [ ]:
#Validation Dataset
pickle_labels = open('/content/drive1/My Drive/Validation_Set_labels1.pickle',"rb")
labels_validation = pickle.load(pickle_labels)
print(np.array(labels_validation).shape)

pickle_values = open('/content/drive1/My Drive/Validation_Set_values1.pickle',"rb")
values_validation = pickle.load(pickle_values)
print(np.array(values_validation).shape)

(3583, 2)
(3583, 8, 2048)


In [ ]:
#Testing Dataset
pickle_labels = open('/content/drive1/My Drive/Testing_Set_labels1.pickle',"rb")
labels_test = pickle.load(pickle_labels)
print(np.array(labels_test).shape)

pickle_values = open('/content/drive1/My Drive/Testing_Set_values1.pickle',"rb")
values_test = pickle.load(pickle_values)
print(np.array(values_test).shape)

(4091, 2)
(4091, 8, 2048)


In [ ]:
values_train=np.concatenate((values_train, values_test), axis=0)
labels_train=np.concatenate((labels_train, labels_test), axis=0)
# values_train=np.concatenate((values_train, values_validation), axis=0)
# labels_train=np.concatenate((labels_train, labels_validation), axis=0)
print(np.array(labels_train).shape)
print(np.array(values_train).shape)

(13599, 2)
(13599, 8, 2048)


In [ ]:
chunk_size = 2048
n_chunks = 8
rnn_size = int((np.array(labels_train).shape[0])/3)
# rnn_size=512

Model = Sequential()

Model.add(LSTM(rnn_size, input_shape=(n_chunks, chunk_size)))

Model.add(Dense(1024, kernel_regularizer=regularizers.l2(0.01)))
Model.add(Activation('relu'))
Model.add(BatchNormalization())
# Model.add(Dropout(0.2))

# Model.add(Dense(400, activation='sigmoid'))

Model.add(Dense(50, activation='sigmoid'))
Model.add(BatchNormalization())
# Model.add(Dropout(0.2))

#Output Layer specifying number of categories
Model.add(Dense(2, activation='softmax'))
Model.add(BatchNormalization())

#Compiling RNN:
# adam=keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)
Model.compile(loss='mean_absolute_error', optimizer='nadam', metrics=['accuracy'])

In [ ]:
epoch = 100
batchS = 100
history = Model.fit(np.array(values_train), np.array(labels_train), epochs=epoch, batch_size=batchS, verbose=2, validation_data = (np.array(values_validation),np.array(labels_validation)))
# history = Model.fit(np.array(values_train), np.array(labels_train), epochs=epoch, batch_size=batchS, verbose=2)

Train on 13599 samples, validate on 3583 samples
Epoch 1/100
 - 176s - loss: 2.1753 - accuracy: 0.7649 - val_loss: 1.3892 - val_accuracy: 0.1998
Epoch 2/100
 - 169s - loss: 0.7564 - accuracy: 0.8913 - val_loss: 0.5697 - val_accuracy: 0.8002
Epoch 3/100
 - 166s - loss: 0.5496 - accuracy: 0.9249 - val_loss: 1.3003 - val_accuracy: 0.1998
Epoch 4/100
 - 171s - loss: 0.5914 - accuracy: 0.8515 - val_loss: 1.1471 - val_accuracy: 0.1998
Epoch 5/100
 - 196s - loss: 0.6071 - accuracy: 0.7552 - val_loss: 0.9308 - val_accuracy: 0.1998
Epoch 6/100


KeyboardInterrupt: ignored

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

In [ ]:
# Evaluating Model
score = Model.evaluate(np.array(values_test), np.array(labels_test), batch_size=batchS, verbose=1)
print(score)

In [ ]:
Model.metrics_names

In [ ]:
ans=Model.predict(np.array(values_test), verbose=1)

In [ ]:
i=0;
count=0;
for res in ans:
  if(res[0]>=res[1]):
    l=1
  else:
    l=0
  if(l==1 and labels_test[i][0]==1) or (l==0 and labels_test[i][1]==0):
    count+=1;
  i+=1;
print((count/i)*100)

In [ ]:
# Saving Model
Model.save("/content/drive1/My Drive/lstmModel.h5", overwrite=True)

In [ ]:
# Saving Training History
pickle_out=open("/content/drive1/My Drive/lstmHistoryDict.pickle","wb")
pickle.dump(history, pickle_out)
pickle_out.close()